# Project_hongo

## Introducción

### 1- Trabajar los datos
transformar los datos de ingreso, a variables de ingreso.

### 2- Diseño de la red
Pensar en la estructura.

### 3- Costo
Definir la función de costo para entrenar la red.

### 4- Definir el entrenamiento
Manera a modificar los parametros (pesos)

### 5- Prueba
entrenamiento y testeo.


# Imports

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf

/home/pablo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load data

In [2]:
data = pd.read_csv("mushrooms.csv")

# Define variables

In [3]:
planedata = []
all_possible_values = []
planedata=np.reshape(np.array(planedata),(len(data),0))

num_values = 0

In [4]:
planedata

array([], shape=(8124, 0), dtype=float64)

# Load data in planedata variable

In [5]:
for atr in data.columns:

    counter = Counter(data[atr])
    parcial_matrix = np.zeros((len(data),len(counter)))

    possible_values_parcial = list(counter.keys())
    all_possible_values += possible_values_parcial

    values = list(data[atr])

    n_reg=0
    for reg in values:
        index = possible_values_parcial.index(reg)
        parcial_matrix[n_reg,index] = 1
        n_reg += 1

    planedata = np.append(planedata,parcial_matrix,axis=1)

In [6]:
labels_data=planedata[:,0:1]
inputs_data=planedata[:,2:len(planedata[0])]

In [7]:
np.shape(inputs_data)


(8124, 117)

# Variable definition

In [8]:
l = 0.01
epoch = 30
batch_size = 20
n_train = int(len(labels_data)*.9)

# Divide data in inputs and test

In [9]:
labels_train=labels_data[0:n_train]
labels_test=labels_data[n_train+1:len(labels_data)]

inputs_train=inputs_data[0:n_train]
inputs_test=inputs_data[n_train+1:len(inputs_data),:]

list_labels_train = []
list_inputs_train = []

num_data = int((len(labels_train)/batch_size))

for i in range (num_data):
    list_inputs_train.append(inputs_train[i*batch_size:(i+1)*batch_size])
    list_labels_train.append(labels_train[i*batch_size:(i+1)*batch_size])

# Initializater Tensor Flow

In [10]:
# Inputs
inputs = tf.placeholder(shape=(None,len(inputs_train[0])), dtype=tf.float32, name="inputs")

# Labels
labels = tf.placeholder(shape=(None,1), dtype=tf.float32, name="labels")

In [11]:
len(inputs_train[0])

117

In [12]:
# Building the model
input_layer = tf.contrib.layers.fully_connected(inputs=inputs, num_outputs = 25, activation_fn = tf.nn.relu)
hidden_layer = tf.contrib.layers.fully_connected(inputs=input_layer, num_outputs = 5, activation_fn = tf.nn.relu)
outputs = tf.contrib.layers.fully_connected(inputs=hidden_layer, num_outputs = 1, activation_fn = tf.nn.sigmoid)

# Initializater TensorFlow
init = tf.global_variables_initializer()

# Cost and Optimizer

In [13]:
with tf.name_scope("Cost_and_Optimizer"):
    cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=outputs, name="cost")) / batch_size
    optimizer = tf.train.GradientDescentOptimizer(l)
    train = optimizer.minimize(cost)

# Train neural network

In [14]:
with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./summaries/lamnda {} epoch {} batch {}".format(l,epoch,batch_size))
    writer.add_graph(sess.graph)
    sess.run(init)

    for i in range(epoch):
        for j in range(num_data):
            sess.run(train, feed_dict={inputs: list_inputs_train[j], labels: list_labels_train[j]})

        #To calcule acurracy and cost
        if i%10==0:
            partial_cost = sess.run(cost, feed_dict={inputs: inputs_train, labels: labels_train})
            result = sess.run(outputs, feed_dict={inputs: inputs_train, labels: labels_train})
            predictions = result > 0.5

            accuracy = np.sum(labels_train == predictions) / len(labels_train)

            print("Train: Epoch N°: {} - Cost = {} - Accuracy = {}%".format(i, partial_cost, accuracy * 100))

#    s = sess.run(merged_summary, feed_dict={inputs: inputs_train, labels: labels_train})
#    writer.add_summary(s, i)

    # Print cost test and final acurracy
    final_cost = sess.run(cost, feed_dict={inputs: inputs_test, labels: labels_test})
    outputs = sess.run(outputs, feed_dict={inputs: inputs_test, labels: labels_test})
    predictions = outputs > 0.5

    final_accuracy = np.sum(labels_test == predictions) / len(labels_test)

print("Test: Cost = {} - Accuracy = {}%".format(final_cost, final_accuracy* 100))

Train: Epoch N°: 0 - Cost = 253.0043182373047 - Accuracy = 52.633018738886605%
Train: Epoch N°: 10 - Cost = 214.8970947265625 - Accuracy = 90.13814799617016%
Train: Epoch N°: 20 - Cost = 201.1691436767578 - Accuracy = 98.30392559157434%
Test: Cost = 20.48482322692871 - Accuracy = 99.26108374384236%
